In [1]:
!pip install selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import requests
import os

# ChromeDriver 경로 설정
chromedriver_path = os.path.join(os.getcwd(), "chromedriver.exe")
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

# 구글 이미지 검색 URL로 이동
query = "beautiful roses"
url = f"https://www.google.com/search?q={query}&tbm=isch"
driver.get(url)

# 저장 디렉토리 설정
save_dir = "downloaded_images4"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 스크롤 다운 반복
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# BeautifulSoup로 HTML 파싱
soup = BeautifulSoup(driver.page_source, 'html.parser')
img_tags = soup.find_all('img')

# 이미지 다운로드
for idx, img_tag in enumerate(img_tags):
    img_url = img_tag.get('src')
    
    # 데이터 URL 건너뛰기
    if img_url and img_url.startswith('data:image/'):
        continue
    
    try:
        img_data = requests.get(img_url).content
        filename = f"image_{idx}.jpg"
        filepath = os.path.join(save_dir, filename)
        with open(filepath, 'wb') as f:
            f.write(img_data)
        print(f"{filename} 다운로드 완료")
    except Exception as e:
        print(f"Error downloading {img_url}: {e}")

driver.quit()

ModuleNotFoundError: No module named 'selenium'

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import csv
import time
import os

# 크롬 드라이버 경로 설정
chromedriver_path = os.path.join(os.getcwd(), "chromedriver.exe")  # 크롬 드라이버 경로를 입력해주세요.

# 크롬 드라이버 서비스 객체 생성
service = Service(chromedriver_path)

# 웹 드라이버 객체 생성
driver = webdriver.Chrome(service=service)

# 크롤링할 URL
url = 'https://www.boannews.com/media/t_list.asp?mkind=0'

# 웹 페이지 열기
driver.get(url)

# 뉴스 데이터를 저장할 리스트
news_data = []

# 크롤링할 페이지 수
num_pages = 2  # 원하는 페이지 수를 입력해주세요.

for page in range(1, num_pages + 1):
    # 현재 페이지의 HTML 소스 코드 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 뉴스 목록 찾기
    news_list = soup.select('.news_list a:not(.news_content)')

    for news in news_list:
        # 뉴스 링크 추출
        news_url = 'https://www.boannews.com' + news['href']

        # 새 탭에서 뉴스 링크 열기
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[-1])
        driver.get(news_url)

        # 뉴스 내용 크롤링
        news_html = driver.page_source
        news_soup = BeautifulSoup(news_html, 'html.parser')

        # 뉴스 제목
        news_title = news_soup.select_one('#news_title02 h1').text.strip()

        # 뉴스 날짜
        news_date = ':'.join(news_soup.select_one('#news_util01').text.strip().split(':')[1:]).strip()

        # 뉴스 내용
        news_content = news_soup.select_one('#news_content').text.strip()

        # 크롤링한 데이터를 리스트에 추가
        news_data.append([news_title, news_date, news_content])

        # 탭 닫기
        driver.close()

        # 원래 탭으로 돌아가기
        driver.switch_to.window(driver.window_handles[0])

    # 다음 페이지로 이동
    next_page_url = f'https://www.boannews.com/media/t_list.asp?Page={page+1}&kind=0'
    driver.get(next_page_url)
    time.sleep(2)  # 2초 대기 (페이지 로딩을 위해)

# 웹 드라이버 종료
driver.quit()


# CSV 파일로 저장
with open('news_data3.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Date', 'Content'])
    writer.writerows(news_data)

print('크롤링이 완료되었습니다.')

ModuleNotFoundError: No module named 'selenium'

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import os
import time

# 크롬 드라이버 경로 설정
chromedriver_path = os.path.join(os.getcwd(), "C:\\Users\\USER\\Documents\\Crawling\\chromedriver.exe")  # 크롬 드라이버 경로를 입력해주세요.

# 크롬 드라이버 서비스 객체 생성
service = Service(chromedriver_path)

# 웹 드라이버 객체 생성
driver = webdriver.Chrome(service=service)

# 블로그 URL
blog_url = 'https://creamerburger.tistory.com'

# 댓글을 단 링크 목록
commented_links = []

# 페이지 번호
page_num = 1

while True:
    # 웹 페이지 열기
    driver.get(f"{blog_url}/?page={page_num}")
    
    # 게시물 링크 추출
    post_links = driver.find_elements(By.CSS_SELECTOR, '.list_content .link_post')
    
    if not post_links:
        break  # 게시물이 없으면 반복 종료
    
    for link in post_links:
        post_url = link.get_attribute('href')
        
        # 새 탭에서 게시물 링크 열기
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[-1])
        driver.get(post_url)
        
        # 댓글 작성 필드 찾기
        name_input = driver.find_element(By.CSS_SELECTOR, 'input[placeholder="이름"]')
        password_input = driver.find_element(By.CSS_SELECTOR, 'input[placeholder="비밀번호"]')
        comment_textarea = driver.find_element(By.CSS_SELECTOR, 'textarea[placeholder="댓글을 달아주세요 :)"]')
        submit_button = driver.find_element(By.CSS_SELECTOR, '.commentWrite input[type="submit"]')
        
        # 댓글 작성
        name_input.send_keys("테스트")
        password_input.send_keys("1234")
        comment_textarea.send_keys("힘내세요!")
        submit_button.click()
        
        time.sleep(2)  # 댓글 작성 후 잠시 대기
        
        # 댓글을 단 링크 목록에 추가
        commented_links.append(post_url)
        
        # 탭 닫기
        driver.close()
        
        # 원래 탭으로 돌아가기
        driver.switch_to.window(driver.window_handles[0])
    
    page_num += 1  # 다음 페이지로 이동

# 웹 드라이버 종료
driver.quit()

# 댓글을 단 링크 목록 출력
print("댓글을 단 링크 목록:")
for link in commented_links:
    print(link)

ModuleNotFoundError: No module named 'selenium'